# Automação Web e Busca de Informações com Python

#### Desafio: 

Trabalhamos em uma importadora e o preço dos nossos produtos é vinculado a cotação de:
- Dólar
- Euro
- Ouro

Precisamos pegar na internet, de forma automática, a cotação desses 3 itens e saber quanto devemos cobrar pelos nossos produtos, considerando uma margem de contribuição que temos na nossa base de dados.

Base de Dados: https://drive.google.com/drive/folders/1KmAdo593nD8J9QBaZxPOG1yxHZua4Rtv?usp=sharing

Para isso, vamos criar uma automação web:

- Usaremos o selenium
- Importante: baixar o webdriver

In [20]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

# criar um navegador

navegador = webdriver.Chrome()

# entrar em sites

#encontrar informações / itens do site

# fechar o navegador

# Passo 1: Pegar a cotação do dólar

navegador.get("https://www.google.com.br/")

navegador.find_element(By.XPATH, 
                       '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div[2]/div[2]/input').send_keys("cotação dolar")
navegador.find_element(By.XPATH, 
                       '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div[2]/div[2]/input').send_keys(Keys.ENTER)

cotacao_dolar = navegador.find_element(By.XPATH, 
                                      '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute("data-value")
print(cotacao_dolar)
# Passo 2: Pegar a cotação do euro

navegador.get("https://www.google.com.br/")

navegador.find_element(By.XPATH, 
                       '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div[2]/div[2]/input').send_keys("cotação euro")
navegador.find_element(By.XPATH, 
                       '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div[2]/div[2]/input').send_keys(Keys.ENTER)

cotacao_euro = navegador.find_element(By.XPATH, 
                                      '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute("data-value")
print(cotacao_euro)

# Passo 3: Pegar a cotação do ouro

navegador.get("https://www.melhorcambio.com/ouro-hoje")

cotacao_ouro = navegador.find_element(By.XPATH, '//*[@id="comercial"]').get_attribute("value")
cotacao_ouro = cotacao_ouro.replace(",", ".")
print(cotacao_ouro)

navegador.quit()


5.5085999999999995
6.318419286
322.31


### Agora vamos atualiza a nossa base de preços com as novas cotações

- Importando a base de dados

In [21]:
# Passo 4: Importar a base de dados e atualizar as cotações na minha base

import pandas as pd

tabela = pd.read_excel("Produtos.xlsx")
display(tabela)


,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5,4999.95,1.40,6999.930
1,Carro Renault,4500.00,Euro,6,27000.00,2.00,54000.000
2,Notebook Dell,899.99,Dólar,5,4499.95,1.70,7649.915
3,IPhone,799.00,Dólar,5,3995.00,1.70,6791.500
4,Carro Fiat,3000.00,Euro,6,18000.00,1.90,34200.000
5,Celular Xiaomi,480.48,Dólar,5,2402.40,2.00,4804.800
6,Joia 20g,20.00,Ouro,350,7000.00,1.15,8050.000


- Atualizando os preços e o cálculo do Preço Final

In [23]:
# Passo 5: Calcular os novos preços e salvar/exportar a base de dados

tabela.loc[tabela["Moeda"] == "Dólar" , "Cotação"] = float(cotacao_dolar)
tabela.loc[tabela["Moeda"] == "Euro" , "Cotação"] = float(cotacao_euro)
tabela.loc[tabela["Moeda"] == "Ouro" , "Cotação"] = float(cotacao_ouro)

# atualizar as colunas

# Preço de Compra = Preço Original * Cotação
tabela["Preço de Compra"] = tabela["Preço Original"] * tabela["Cotação"]

# Preço de Venda = Preço de Compra * Margem
tabela["Preço de Venda"] = tabela["Preço de Compra"] * tabela["Margem"]

display(tabela)


,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5.508600,5508.544914,1.40,7711.962880
1,Carro Renault,4500.00,Euro,6.318419,28432.886787,2.00,56865.773574
2,Notebook Dell,899.99,Dólar,5.508600,4957.684914,1.70,8428.064354
3,IPhone,799.00,Dólar,5.508600,4401.371400,1.70,7482.331380
4,Carro Fiat,3000.00,Euro,6.318419,18955.257858,1.90,36014.989930
5,Celular Xiaomi,480.48,Dólar,5.508600,2646.772128,2.00,5293.544256
6,Joia 20g,20.00,Ouro,322.310000,6446.200000,1.15,7413.130000


### Agora vamos exportar a nova base de preços atualizada

In [25]:
tabela.to_excel("Produtos Novo.xlsx")